<a href="https://colab.research.google.com/github/pavansai26/ADAGRAD-OPTIMIZER-IMPLEMENTATION-FROM-SCRATCH/blob/master/ADAGRAD_OPTIMIZER_IMPLEMENTATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$\begin{align}
\mathbf{w}_{t+1} &amp;= \mathbf{w}_{t} - \frac{\eta}{\sqrt{G_{t} + \epsilon}} \odot g_{t}\\
&amp;= \mathbf{w}_{t} - \frac{\eta}{\sqrt{G_{t} + \epsilon}} \odot \frac{\partial \mathcal{L}(\mathbf{w}_{t})}{\partial \mathbf{w}_{t}}
\end{align}$$

$G_{t}$: the sum of the squares of the past gradients w.r.t. to all parameters $\mathbf{w}$ along its diagonal.

where $G_{t} \in \mathbb{R}^{d \times d}$ here is a diagonal matrix 

where each diagonal element $i,i$ is the sum of the squares of the gradients w.r.t. $\mathbf{w}_{i}$ up to time step $t$,

while $\epsilon$ is a smoothing term that avoids division by zero

# implementation of the algorithm

importing the libraries

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm

Beale function

$$ f(x, y) = (1.5 - x + xy)^{2} + (2.25 - x + xy^{2})^{2} + (2.625 - x +xy^{3})^{2}$$

analytic solution (global minima)
$(x, y) = (3, 0.5)$

In [ ]:
f = lambda x, y: (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2

In [ ]:
def gradients(x, y):
  """Gradient of Beale function.

  Args:
    x: x-dimension of inputs
    y: y-dimension of inputs

  Returns:
    grads: [dx, dy], shape: 1-rank Tensor (vector) np.array
      dx: gradient of Beale function with respect to x-dimension of inputs
      dy: gradient of Beale function with respect to y-dimension of inputs
  """
  dx = 2. * ( (1.5 - x + x * y) * (y - 1) + \
                (2.25 - x + x * y**2) * (y**2 - 1) + \
                (2.625 - x + x * y**3) * (y**3 - 1) )
  dy = 2. * ( (1.5 - x + x * y) * x + \
              (2.25 - x + x * y**2) * 2. * x * y + \
              (2.625 - x + x * y**3) * 3. * x * y**2 )
  grads = np.array([dx, dy])
  return grads

Build a Optimizer

In [ ]:
class AdagradOptimizer():
  def __init__(self, function, gradients, x_init=None, y_init=None, learning_rate=0.01, initial_accumulator_value=0.1):
    self.f = function
    self.g = gradients
    scale = 3.0
    self.vars = np.zeros([2])

    if x_init is not None:
      self.vars[0] = x_init
    else:
      self.vars[0] = np.random.uniform(low=-scale, high=scale)

    if y_init is not None:
      self.vars[1] = y_init
    else:
      self.vars[1] = np.random.uniform(low=-scale, high=scale)
    
    print("x_init: {:.3f}".format(self.vars[0]))
    print("y_init: {:.3f}".format(self.vars[1]))

    self.lr = learning_rate
    self.grads_squared = np.zeros([2])
    self.grads_squared.fill(initial_accumulator_value)
    self.epsilon = 1e-7

    # for accumulation of loss and path (w, b)
    self.z_history = []
    self.x_history = []
    self.y_history = []

  def func(self, variables):
    """Beale function.
    
    Args:
      variables: input data, shape: 1-rank Tensor (vector) np.array
        x: x-dimension of inputs
        y: y-dimension of inputs
      
    Returns:
      z: Beale function value at (x, y)
    """
    x, y = variables
    z = self.f(x, y)
    return z
  def gradients(self, variables):
    """Gradient of Beale function.
    
    Args:
      variables: input data, shape: 1-rank Tensor (vector) np.array
        x: x-dimension of inputs
        y: y-dimension of inputs
      
    Returns:
      grads: [dx, dy], shape: 1-rank Tensor (vector) np.array
        dx: gradient of Beale function with respect to x-dimension of inputs
        dy: gradient of Beale function with respect to y-dimension of inputs
    """
    x, y = variables
    grads = self.g(x, y)
    return grads
  
  def weights_update(self, grads):
    """Weights update using adagrad.
    grads2 = grads2 + grads**2
    w' = w - lr * grads / (sqrt(grads2) + epsilon)"""
    self.grads_squared = self.grads_squared + grads**2
    self.vars = self.vars - self.lr * grads / (np.sqrt(self.grads_squared) + self.epsilon)

  def history_update(self, z, x, y):
    """Accumulate all interesting variables
    """
    self.z_history.append(z)
    self.x_history.append(x)
    self.y_history.append(y)

  def train(self, max_steps):
    pre_z = 0.0
    print("steps: {}  z: {:.6f}  x: {:.5f}  y: {:.5f}".format(0, self.func(self.vars), self.x, self.y))

    file = open('adagrad.txt', 'w')
    file.write("{:.5f}  {:.5f}\n".format(self.x, self.y))

    for step in range(max_steps):  
      self.z = self.func(self.vars)
      self.history_update(self.z, self.x, self.y)

      self.grads = self.gradients(self.vars)
      self.weights_update(self.grads)
      file.write("{:.5f}  {:.5f}\n".format(self.x, self.y))
      
      if (step+1) % 100 == 0:
        print("steps: {}  z: {:.6f}  x: {:.5f}  y: {:.5f}  dx: {:.5f}  dy: {:.5f}".format(step+1, self.func(self.vars), self.x, self.y, self.dx, self.dy))

      if np.abs(pre_z - self.z) < 1e-7:
        print("Enough convergence")
        print("steps: {}  z: {:.6f}  x: {:.5f}  y: {:.5f}".format(step+1, self.func(self.vars), self.x, self.y))
        self.z = self.func(self.vars)
        self.history_update(self.z, self.x, self.y)
        break
      
      pre_z = self.z
    file.close()
  
    self.x_history = np.array(self.x_history)
    self.y_history = np.array(self.y_history)
    self.path = np.concatenate((np.expand_dims(self.x_history, 1), np.expand_dims(self.y_history, 1)), axis=1).T
  
  @property
  def x(self):
    return self.vars[0]
  
  @property
  def y(self):
    return self.vars[1]
  
  @property
  def dx(self):
    return self.grads[0]
  
  @property
  def dy(self):
    return self.grads[1]

    
